Administrative Boundary Indonesia (https://imcarto.webflow.io/gdb)

In [1]:
import os

import pandas as pd
import geopandas as gpd

import folium

### Load Boundaries Geo-Location

In [2]:
path = "F:\Data\datas\geo"
Indo_kec_shp_pathfile = os.path.join(path, "batas_kec", "batas_kec.shp")

In [3]:
gdf_Indo_kec = gpd.read_file(Indo_kec_shp_pathfile)

In [4]:
gdf_Indo_kec_jakarta = gdf_Indo_kec.query("NAME_1 == 'Jakarta Raya'").copy()
gdf_Indo_kec_jakarta.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
gdf_Indo_kec_jakarta = gdf_Indo_kec_jakarta.reset_index(drop=True)
gdf_Indo_kec_jakarta['cent_lat'] = gdf_Indo_kec_jakarta.geometry.centroid.y
gdf_Indo_kec_jakarta['cent_lon'] = gdf_Indo_kec_jakarta.geometry.centroid.x

F:\Users\work\AppData\Local\Temp\ipykernel_13016\86598428.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_Indo_kec_jakarta['cent_lat'] = gdf_Indo_kec_jakarta.geometry.centroid.y
F:\Users\work\AppData\Local\Temp\ipykernel_13016\86598428.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_Indo_kec_jakarta['cent_lon'] = gdf_Indo_kec_jakarta.geometry.centroid.x


In [6]:
df_ispa_dki_all = pd.read_csv("df_ispa_dki_all.csv")

In [7]:
df_ispa_dki_all_geo = df_ispa_dki_all.merge(
    gdf_Indo_kec_jakarta[['NAME_3', 'geometry']], 
    left_on='sub_district', right_on='NAME_3', how='left'
).drop("NAME_3", axis=1)

In [8]:
df_ispa_dki_all_geo = gpd.GeoDataFrame(df_ispa_dki_all_geo)

In [ ]:
df_ispa_dki_all_geo.query("year==2015 & month=='Jan'").explore()

### Folium Map

In [54]:
from branca.colormap import linear

In [ ]:
['YlOrBr_05', 'YlOrRd_04', 'YlOrRd_05', 'Reds_09', 'Reds_05', 'YlOrRd_08', 'magma']

In [88]:
colormap = linear.YlOrRd_08.scale(
    df_ispa_dki_all_geo["pm10"].min(), 
    df_ispa_dki_all_geo["pm10"].max()
)
colormap.caption = "pm10 scale"

In [43]:
map_category_color = {
    'SEDANG': 'lightblue', 
    'BAIK': 'green', 
    'TIDAK SEHAT': '#ffff00',
    'SANGAT TIDAK SEHAT': 'tomato',
    'BERBAHAYA': 'black'
}

In [103]:
m = folium.Map([-6.2, 107], zoom_start=10)
popup = folium.GeoJsonPopup(fields=["station", "pm10", "pm25", "o3"])

folium.GeoJson(
    df_ispa_dki_all_geo.query("date=='2015-02-25'"),
    name="pm10",
    control=True,
    zoom_on_click=True,
    style_function=lambda feature: {
        # "fillColor": map_category_color[feature['properties']['category']],
        "fillColor": colormap(feature['properties']['pm10']),
        "color": "black",
        "weight": 2,
        "dashArray": "5, 5",
        "fillOpacity": 0.9,
    },
    highlight_function=lambda feature: {"fillColor": "#ffff00"},
    popup=popup,
    popup_keep_highlighted=True,
    show=False
).add_to(m)
colormap.add_to(m)
folium.LayerControl().add_to(m)

In [104]:
m

In [51]:
df_ispa_dki_all_geo.query("date=='2015-02-25'")

,date,station,pm10,pm25,so2,co,o3,no2,max,critical,category,year,quarter,month,day,day_of_week,district,sub_district,geometry
7213,2015-02-25,DKI2 (Kelapa Gading),60.0,NaN,8.0,36.0,86.0,15.0,86.0,O3,SEDANG,2015,1,Feb,25,Wed,Jakarta Utara,Kelapa Gading,"POLYGON ((106.90662 -6.18064, 106.90655 -6.180..."
7214,2015-02-25,DKI3 (Jagakarsa),22.0,NaN,12.0,45.0,121.0,11.0,121.0,O3,TIDAK SEHAT,2015,1,Feb,25,Wed,Jakarta Selatan,Jagakarsa,"POLYGON ((106.79466 -6.36492, 106.79464 -6.364..."
7215,2015-02-25,DKI1 (Bunderan HI),15.0,NaN,7.0,24.0,56.0,18.0,56.0,O3,SEDANG,2015,1,Feb,25,Wed,Jakarta Pusat,Menteng,"POLYGON ((106.84905 -6.20754, 106.84895 -6.207..."
7216,2015-02-25,DKI5 (Kebon Jeruk),69.0,NaN,3.0,25.0,62.0,8.0,69.0,PM10,SEDANG,2015,1,Feb,25,Wed,Jakarta Barat,Kebonjeruk,"POLYGON ((106.77711 -6.21614, 106.7771 -6.2162..."
7217,2015-02-25,DKI4 (Lubang Buaya),85.0,NaN,20.0,43.0,98.0,11.0,98.0,O3,SEDANG,2015,1,Feb,25,Wed,Jakarta Timur,Cipayung,"POLYGON ((106.9181 -6.34388, 106.91785 -6.3438..."
